# Leveraging Vespa and LLM for Enhanced Culinary and Retail Experiences: AI-Driven Recipe Search and Recommendations

![Author](https://img.shields.io/badge/Author-Soufiane%20AAZIZI-brightgreen)
[![Medium](https://img.shields.io/badge/Medium-Follow%20Me-blue)](https://medium.com/@aazizi.soufiane)
[![GitHub](https://img.shields.io/badge/GitHub-Follow%20Me-lightgrey)](https://github.com/aazizisoufiane)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect%20with%20Me-informational)](https://www.linkedin.com/in/soufiane-aazizi-phd-a502829/)

---

The primary objective of this notebook is to interact with a Language Model (LLM) to inquire about a recipe. The LLM will respond with the recipe's ingredients and cooking instructions. However, the key highlight is that it will generate recommendations for each ingredient from our store using Vespa.ai, a retrieval system. This means that for each ingredient, it will suggest related products or items that you might want to consider when preparing the recipe.

**Example:**
Suppose you ask the LLM for a recipe for "Spaghetti Bolognese." The LLM will not only provide you with the list of ingredients and cooking instructions but also suggest products like pasta, ground beef, tomatoes, and spices that you may need to purchase for making the dish. This recommendation is based on the available products in our store, enhancing your cooking experience.

### [Preprocess Data](#preprocess-data)
### [Build and Deploy Vespa](#build-deploy-vespa)
  - #### [Export the model to ONNX format](#export-onnx)
  - #### [Application package](#application-package)
### [LangChain](#LangChain)
  - #### [Config VespaRetriever](#config-vespa-retriever)
  - #### [Import LangChain tools](#import-lanchain-tools)
  - #### [Function Explanation: build_chain()](#function-explanation)
  - #### [Overall Purpose](#overall-purpose)
  - #### [Key Components](#key-components)
  - #### [Workflow](#workflow)
  - #### [Recipe Recommendation and Ingredient Listing](#Recipe-Recommendation)
  - #### [Input and Processing](#Input-and-Processing)
  - #### [Recommendation Engine](#Recommendation-Engine)
  - #### [Recipe Instructions](#Recipe-Instructions)
  - #### [Interaction and Exit](#Interaction-and-Exit)


In [55]:
%load_ext lab_black

# Preprocess Data <a class="anchor" id="preprocess-data"></a>

In [26]:
import pandas as pd
import json

In [27]:
application_name = "productsContent"

In [28]:
def dataframe_to_vespa_format(input_df, output_file, application_name):
    """
    Convert a Pandas DataFrame to Vespa-compatible JSONL format and write it to a file.

    Args:
        input_df (pandas.DataFrame): The DataFrame to convert.
        output_file (str): The path to the output JSONL file.
        application_name (str): The name of the Vespa application.

    Returns:
        None

    Explanation:
    This function takes a Pandas DataFrame, extracts relevant data, and converts it into Vespa-compatible JSONL format.
    It then writes the JSONL records to a specified output file.

    - input_df (pandas.DataFrame): The input DataFrame containing data to be converted.
    - output_file (str): The path to the output JSONL file where Vespa-compatible data will be saved.
    - application_name (str): The name of the Vespa application.

    For each row in the DataFrame, the function:
    1. Extracts the 'id', 'Price', 'title', and 'description' columns.
    2. Constructs a JSON record in the Vespa format.
    3. Writes the JSON record to the output JSONL file.

    Note:
    - The 'Price' column is converted to a string to maintain data consistency.
    - The 'body' field is constructed by combining 'title', 'description', and 'Price'.

    Example Usage:
    dataframe_to_vespa_format(my_dataframe, "output.jsonl", "my_vespa_app")
    """
    with open(output_file, "w") as jsonl_file:
        for index, row in input_df.iterrows():
            id_value = row["id"]
            price_value = str(row["Price"])
            body_value = (
                "\nName: "
                + row["title"]
                + " \nDescription: "
                + row["description"]
                + " \nPrice: "
                + price_value
            )
            json_record = {
                "put": f"id:{application_name}:{application_name}::{id_value}",
                "fields": {
                    "Price": price_value,
                    "title": row["title"],
                    "description": row["description"],
                    "body": body_value,
                },
            }
            jsonl_file.write(json.dumps(json_record) + "\n")


In [29]:
products = pd.read_csv("retail_product.csv")
dataframe_to_vespa_format(products, "produits.jsonl", application_name)

# Build and Deploy Vespa <a class="anchor" id="build-deploy-vespa"></a>

In this section, we need to build a Vespa application. Before that, we start by installing Docker locally. You can install [Docker Desktop for Mac/Windows](https://docs.docker.com/engine/install/).
For deepdive into vespa [Vespa](https://docs.vespa.ai/en/getting-started.html)

### Export the model to ONNX format <a class="anchor" id="export-onnx"></a>

Vespa offers flexibility in using various embedding methods, including the [ONNX format](https://docs.vespa.ai/en/embedding.html#onnx-export), which is compatible with both the Bert embedder and the Huggingface embedder. In this case, we've chosen to use the [Huggingface embedder](https://docs.vespa.ai/en/embedding.html#huggingface-embedder) with the [all-MiniLM-L6-v2 model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

To set up the embedding, you can follow these steps:

1. Export the ONNX model:

    ```bash
    sudo python export_model_from_hf.py --hf_model sentence-transformers/all-MiniLM-L6-v2 --output_dir all-MiniLM-L6-v2
    ```

2. Debug ONNX models using Vespa's built-in tools. Refer to the [Vespa documentation](https://docs.vespa.ai/en/embedding.html#onnx-export) for more details on this step.

    ```bash
    docker run -v `pwd`:/w \
      --entrypoint /opt/vespa/bin/vespa-analyze-onnx-model \
      vespaengine/vespa \
      /w/sentence-transformers-all-MiniLM-L6-v2.onnx
    ```


### Application package <a class="anchor" id="application-package"></a>

In [18]:
from vespa.package import ApplicationPackage
from vespa.package import *
from pathlib import Path

Create an application package from scratch. it will create all  Vespa configuration files

In [30]:
app_package = ApplicationPackage(name=application_name)

After generating and debugging our ONNX model, we can integrate it into our Vespa application

In [31]:
app_package = ApplicationPackage(
    name=application_name,
    components=[
        Component(
            # id="e5-small-q",
            id="e5",
            type="hugging-face-embedder",
            parameters=[
                Parameter(
                    "transformer-model",
                    {
                        "path": "model/all-MiniLM-L6-v2/sentence-transformers-all-MiniLM-L6-v2.onnx"
                    },
                ),
                Parameter(
                    "tokenizer-model", {"path": "model/all-MiniLM-L6-v2/tokenizer.json"}
                ),
            ],
        )
    ],
)

In our JSON file, we've defined specific keys that we will use to **populate Vespa fields** for our Vespa application. 

Before we embark on this journey, it's crucial to grasp the significance of the **BM25 ranking function**—an integral part of our Vespa application. 

**BM25** is more than just a ranking function; it's the magic wand of information retrieval. It meticulously gauges the relevance of documents to search queries by delving into factors like term frequency, inverse document frequency, and document length. 


Additionally, we employ the power of **indexing**. These indexes are the treasure maps of Vespa, created for specific fields. They're the superhighways of data retrieval, ensuring swift and efficient searches. 

For a deeper dive into indexing, consult the [Vespa documentation](https://docs.vespa.ai/en/reference/schema-reference.html#indexing), where you'll find the keys to unlocking even more potential.

**HNSW**, short for Hierarchical Navigable Small World, is a data structure and algorithm employed in approximate nearest neighbor search. It excels at efficiently locating data points in high-dimensional spaces by constructing a hierarchical graph structure, facilitating rapid and scalable similarity searches. HNSW is commonly utilized in machine learning and data retrieval applications where the speedy identification of similar data points is essential, such as recommendation systems and image recognition.

**Angular distance** is a measure of the shortest separation or difference between two points, typically in the context of spherical coordinates or geographic locations. It is commonly used to calculate the shortest path or great-circle distance between two points on the surface of a sphere, such as measuring the distance between two locations on the Earth's surface. Angular distance is typically expressed in degrees, radians, or other angular units, taking into account the curvature of the surface, and is essential in geographic applications for determining the shortest route or distance between two points on a spherical object like the Earth.

**tensor**: This indicates that the tensor contains floating-point values. It's a way of specifying the data type of the elements within the tensor.


In [63]:
app_package.schema.add_fields(
    Field(name="id", type="int", indexing=["attribute", "summary"]),
    Field(
        name="body",
        type="string",
        indexing=["index", "summary"],
        index="enable-bm25",
        bolding=True,
    ),
    Field(
        name="title",
        type="string",
        indexing=["index", "summary"],
        index="enable-bm25",
        bolding=True,
    ),
    Field(
        name="description",
        type="string",
        indexing=["index", "summary"],
        index="enable-bm25",
        bolding=True,
    ),
    Field(name="Price", type="string", indexing=["attribute", "summary"]),
    Field(
        name="title_embeddings",
        type="tensor<float>(x[384])",
        indexing=["input title", "embed", "index", "attribute"],
        ann=HNSW(distance_metric="angular"),
        is_document_field=False,
    ),
    Field(
        name="description_embeddings",
        type="tensor<float>(x[384])",
        indexing=["input description", "embed", "index", "attribute"],
        ann=HNSW(distance_metric="angular"),
        is_document_field=False,
    ),
)

In [64]:
app_package.schema.add_field_set(
    FieldSet(name="default", fields=["title", "description", "body"])
)


We have defined a hybrid rank profile that takes into account both semantic search and BM25, combining them to determine the relevance of search results.

**First Phase Formula**

In the first phase of ranking, the formula is as follows:

```markdown
first_phase = 100 * closeness(title_embeddings) + 50 * closeness(description_embeddings) + 0.5 * bm25(body)


In [65]:
app_package.schema.add_rank_profile(
    RankProfile(name="bm25", first_phase="bm25(title) + 0.5 * bm25(description)")
)


app_package.schema.add_rank_profile(
    RankProfile(
        name="hybrid",
        inputs=[("query(q)", "tensor<float>(x[384])")],
        inherits="default",
        first_phase="100*closeness(title_embeddings) + 50*closeness(description_embeddings)  + 0.5*bm25(body)",
        match_features=[
            "closeness(title_embeddings) \
        closeness(title_embeddings)  \
        bm25(body)"
        ],
    )
)

We export the application package to the disk before deployment


In [66]:
app_package.schema.add_document_summary(
    DocumentSummary(
        name="all",
        summary_fields=[
            Summary("id", "int"),
            Summary("title", "string"),
            Summary("description", "string"),
            Summary("Price", "string"),
        ],
    )
)

Path("pkg").mkdir(parents=True, exist_ok=True)
app_package.to_files("pkg")

Deploy `app_package` on the local machine using Docker, without leaving the notebook, by creating an instance of `VespaDocker`. You can use this command line:

```python
vespa_docker = vespa_docker.from_container_name_or_id(application_name)


In [67]:
from vespa.deployment import Vespa

app = Vespa("http://localhost:8080/")

In [17]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker(port=8080)
app = vespa_docker.deploy_from_disk(
    application_name=application_name, application_root="pkg"
)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Finished deployment.


Now that we've deployed our Vespa application successfully, we need to install [VESPA CLI](https://docs.vespa.ai/en/vespa-cli.html) and use it to feed our data.

In [40]:
# Use endpoints on localhost
! vespa config set target local

In [41]:

# Display a periodic summary every 3 seconds while feeding:
! vespa  feed --progress=3 produits.jsonl

{
  "feeder.seconds": 173.721,
  "feeder.ok.count": 20262,
  "feeder.ok.rate": 116.635,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 20262,
  "http.request.bytes": 4470241,
  "http.request.MBps": 0.026,
  "http.exception.count": 0,
  "http.response.count": 20262,
  "http.response.bytes": 2368696,
  "http.response.MBps": 0.014,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 248,
  "http.response.latency.millis.avg": 9068,
  "http.response.latency.millis.max": 11715,
  "http.response.code.counts": {
    "200": 20262
  }
}


Check the number of feeded documents

In [68]:
app.query(body={"yql": "select * from sources * where true"}).number_documents_indexed

20262

Make a simple query to test our application

In [69]:
keyword = "olive  1 tablespoon"

result = app.query(
    body={
        "yql": "select * from productsContent_content where userQuery() \
        or ({targetHits:1}nearestNeighbor(title_embeddings,q)) \
        or ({targetHits:1}nearestNeighbor(description_embeddings,q)) \
        ",
        "input.query(q)": f"embed({keyword})",
        "query": f"{keyword}",
        "ranking.profile": "hybrid",
        "bolding": False,
        "hits": 5,
    }
)
result.hits

[{'id': 'id:productsContent:productsContent::13714',
  'relevance': 74.40122045460956,
  'source': 'productsContent_content',
  'fields': {'matchfeatures': {'bm25(body)': 0.0,
    'closeness(title_embeddings)': 0.5283060204679746},
   'sddocname': 'productsContent',
   'description': '1.5 l can',
   'body': '\nName: extra organic virgin olive oil \nDescription: 1.5 l can \nPrice: 27.9195',
   'title': 'extra organic virgin olive oil',
   'documentid': 'id:productsContent:productsContent::13714',
   'Price': '27.9195'}},
 {'id': 'id:productsContent:productsContent::2171',
  'relevance': 66.18247444140717,
  'source': 'productsContent_content',
  'fields': {'matchfeatures': {'bm25(body)': 0.0,
    'closeness(title_embeddings)': 0.43010505946507765},
   'sddocname': 'productsContent',
   'description': '130 g cheese',
   'body': '\nName: goat cheese 1/2 sec \nDescription: 130 g cheese \nPrice: 2.52',
   'title': 'goat cheese 1/2 sec',
   'documentid': 'id:productsContent:productsContent::

# LangChain <a class="anchor" id="LangChain"></a>

In [1]:
import sys

from dotenv import load_dotenv, find_dotenv
from langchain.agents import AgentType
from langchain.retrievers.vespa_retriever import VespaRetriever
from vespa.application import Vespa

# Load environment variables from the local .env file, including OPENAI_API_KEY
_ = load_dotenv(find_dotenv())

MAX_HISTORY_LENGTH = 5

### Config VespaRetriever <a class="anchor" id="config-vespa-retriever"></a>

In [22]:
app = Vespa(url="http://localhost", port=8080)
VespaRetriever.update_forward_refs(Vespa=Vespa)  # We need this line on Jupyter notebook

### Import LangChain tools <a class="anchor" id="import-lanchain-tools"></a>

In [6]:
from langchain.agents import initialize_agent

from langchain.chains import SimpleSequentialChain

from typing import Type, List, Union
from pydantic import BaseModel, Field, validator
from langchain.tools import BaseTool
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import PydanticOutputParser

In [7]:
# Set up our language model (LLM) with the specified parameters

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0.9)

In [8]:
class IngredientItem(BaseModel):
    name: str
    quantity: str
    

class Recipe(BaseModel):
    ingredients: List[IngredientItem]
    instruction: List


class Recommendation(BaseModel):
    name: str
    description: str
    price: Union[float, None]
    
    @validator("price", pre=True, always=True)
    def handle_null_price(cls, value):
        # Convert "null" to None or set a default value if needed
        if value is None or value == "null":
            return None  # Change "null" to None
        try:
            return float(value)
        except (TypeError, ValueError):
            return None  # Handle invalid values gracefully



class Recs(BaseModel):
    recommendations: List[Recommendation] = Field(description="List of recommendations")




In [9]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKBOLDBLUE = '\033[1;34m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    OKYELLOW = '\033[33m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [10]:
# Create an instance of PydanticOutputParser with the 'Recs' Pydantic model
parser_extraction = PydanticOutputParser(pydantic_object=Recs)

# Create another instance of PydanticOutputParser with the 'Recipe' Pydantic model
parser_ingredient = PydanticOutputParser(pydantic_object=Recipe)


## Function Explanation: build_chain() <a class="anchor" id="function-explanation"></a>

The function `build_chain()` is responsible for constructing a processing chain designed to list the ingredients required for a recipe. Below is an explanation of the function's purpose and its inner workings:

**Function Purpose:** <a class="anchor" id="Function-Purpose"></a>
The primary objective of this function is to create a processing chain for culinary tasks. Specifically, it aims to provide a list of ingredients needed to prepare a given recipe. The function returns an `LLMChain` object, which represents a chain of processing steps for handling recipe-related information.

**Function Details:** <a class="anchor" id="Function-Details"></a>

- **Template Definition:** <a class="anchor" id="Template-Definition"></a> Inside the function, there's a multi-line string named `recipe_ingredients_template`. This string serves as a template for creating a message prompt that will be presented to the language model. It outlines the mission for the culinary expert using placeholders like `{recipe}` and `{format_instructions}`.

- **Format Instructions:** The function retrieves format instructions from the `parser_ingredient` object by calling `parser_ingredient.get_format_instructions()`. These instructions are used within the message prompt template.

- **Human Message Prompt Template:** The function defines a `human_message_prompt` using the `HumanMessagePromptTemplate`. It specifies the message prompt template based on `recipe_ingredients_template`. It expects an input variable named "recipe" and provides partial variables, including "format_instructions."

- **Chat Prompt Template:** A `chat_prompt_template` is created using the `ChatPromptTemplate` and is based on the `human_message_prompt`.

- **LLMChain Creation:** The function then constructs an `LLMChain` named `llm_chain`. This chain utilizes a language model (`llm`) and the chat prompt template.

- **Return:** Finally, the function returns the `LLMChain` object `llm_chain` as its result.

In summary, this function sets up a chain of processing steps for generating precise ingredient lists for various recipes. It utilizes a language model, agent, and message prompts to encapsulate the entire information extraction process and returns the processing chain for further use in culinary-related tasks.


In [11]:

# Define a docstring for the function 'build_chain'
def build_chain():
    """
    Build a chain for listing ingredients of a recipe.

    Returns:
        LLMChain: A chain for processing recipe-related information.
    """
    recipe_ingredients_template = """ You are a culinary expert known for your meticulous recipe analysis.
    Your task is to list all the ingredients required for a specific recipe. 
    You have a vast knowledge of recipes from around the world and can provide precise ingredient lists for various dishes.

    Here's your challenge:
    Given the name of a recipe, list all the ingredients needed to prepare that dish. Include exact quantities and instruction.

    Recipe Name: {recipe}

    {format_instructions}
    """

    # Retrieve format instructions from the PydanticOutputParser
    format_instructions = parser_ingredient.get_format_instructions()

    # Define a human message prompt template
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=recipe_ingredients_template,
            input_variables=["recipe"],
            partial_variables={"format_instructions": format_instructions},
        )
    )

    # Create a chat prompt template
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

    # Create an LLMChain with the language model
    return LLMChain(llm=llm, prompt=chat_prompt_template, verbose=False)



**Note**: We can consider other templates and use RouterChain by adapting our previous example. Here are some additional templates that we can add or replace with the previous one:

In [16]:
healthy_recipe_template = """You are a health-conscious culinary expert known for crafting nutritious and delicious recipes. 
Your mission is to provide a list of ingredients and instructions for a healthy and balanced meal. Whether it's a salad, smoothie,
or low-calorie dish, you're here to guide users toward healthier choices.

Recipe Name: {3recipe}

{format_instructions}
"""
vegetarian_recipe_template = """As a vegetarian culinary expert, you specialize in meat-free recipes that are both satisfying and delicious. 
Your task is to compile a list of ingredients and detailed instructions for a vegetarian dish, ensuring it's both flavorful and satisfying.

Recipe Name: {recipe}

{format_instructions}
"""
vegan_recipe_template = """You are a dedicated vegan chef, passionate about creating plant-based recipes. 
Your challenge is to provide a comprehensive list of ingredients and step-by-step instructions for a vegan-friendly dish. 
Your goal is to help users enjoy a cruelty-free and environmentally conscious meal.

Recipe Name: {recipe}

{format_instructions}
"""
gluten_free_recipe_template = """As a gluten-free cooking expert, you excel in crafting recipes suitable for individuals with gluten sensitivities or allergies. 
Your mission is to outline the ingredients and preparation steps for a gluten-free dish that is safe and delicious for those with specific dietary 
requirements.

Recipe Name: {recipe}

{format_instructions}
"""
low_carb_recipe_template = """You specialize in low-carb cooking, helping individuals maintain a reduced carbohydrate intake. Your task is to present a 
list of ingredients and cooking instructions for a low-carb recipe that is both satisfying and suitable for those following a low-carb lifestyle.

Recipe Name: {recipe}

{format_instructions}
"""


### Overall Purpose <a class="anchor" id="overall-purpose"></a>

**Objective**: The primary purpose of the `build_extraction_chain()` function is to establish a processing chain tailored to the task of identifying the most suitable ingredients available in our store. This process aims to construct a recommendation engine that provides recommendations for each ingredient based on its availability and suitability for various recipes.
### Key Components <a class="anchor" id="key-components"></a>

1. **Search Function**: The notebook includes a `search` function that leverages Vespa, a search engine, to retrieve relevant product information based on user queries.

2. **Ingredient Price Tool**: A custom class named `IngredientPriceTool` is defined to provide product suggestions and identify optimal options with the lowest prices based on ingredient names and quantities.

3. **Agent Creation**: The `build_agent` function is responsible for creating a culinary agent, equipped with tools like the `IngredientPriceTool`, to manage culinary-related tasks.

4. **Extraction Chain**: The `build_extraction_chain` function constructs an extraction chain designed for culinary tasks. It combines a language model and the culinary agent to extract various types of information, including recommendations, ingredient lists, and cooking instructions.

### Workflow <a class="anchor" id="workflow"></a>

1. Users interact with the notebook by inputting queries related to recipes and culinary tasks.

2. The `search` function is employed to process user queries and retrieve relevant product information using Vespa.

3. A culinary agent, created using the `build_agent` function, assists in providing ingredient suggestions and handling culinary tasks.

4. The `build_extraction_chain` function manages the extraction of culinary information by utilizing the language model and culinary agent.

5. Users receive responses that include ingredient lists, product recommendations, and cooking instructions based on their queries.

**Note:** As we currently cannot use our [VespaRetriever](https://python.langchain.com/docs/integrations/retrievers/vespa) in its current form since it only overrides the query parameter, the temporary solution is to initialize a new retriever for every query. This will be the approach until it is fixed by the Langchain/Vespa teams. To streamline this process, we will create an agent that will handle the search operations on our behalf.


In [12]:
# Define a docstring for the function 'search'
def search(query, hits=100):
    """
    Perform a search query using Vespa.

    Args:
        query (str): The search query.
        hits (int): The number of search results to retrieve.

    Returns:
        list: A list of relevant documents.
    """
    yql = {
        "yql": "select * from productsContent_content where userQuery() \
        or ({targetHits:1}nearestNeighbor(title_embeddings,q)) \
        or ({targetHits:1}nearestNeighbor(description_embeddings,q)) \
        ",
        "input.query(q)": f"embed({query})",
        "query": f"{query}",
        "ranking.profile": "hybrid",
        "bolding": False,
        "hits": hits,
    }

    # Create a Vespa retriever
    retriever = VespaRetriever(
        app=app,
        body=yql,
        content_field="body",
        metadata_fields=["Price"],
        # metadata={"body": str},
    )

    # Get relevant documents based on the query
    docs = retriever.get_relevant_documents(query)
    return [d.page_content for d in docs]

class IngredientInput(BaseModel):
    """Inputs for IngredientTool"""

    ingredient_name: str = Field(description="Ingredient name for recipe")
    quantity: str = Field(description="quantity of ingredient for recipe")


class IngredientPriceTool(BaseTool):
    name = "IngredientTool"
    description = """ Please provide a list of product suggestions from the catalog based on the ingredients listed in the line. 
    Ensure that each product has the optimal quantity and the lowest price. If no exact match is found for an ingredient, please use 'MISSING'. 
    For multiple ingredients, search for the best product option for each individual ingredient.

    Here is the output schema:
    ```
    {"properties": {"ingredient": 
        {
            "title": "Ingredient", "description": "Provide suggestions", "type": "array", "items": {}}, 
            "required": ["ingredient"]
            }
    }
    ```

    """
    # provide answer as: "\nProduct:   \Quantity:    \Instruction:

    args_schema: Type[BaseModel] = IngredientInput

    def _run(self, ingredient_name: str, quantity: str):
        price_response = search(ingredient_name + quantity)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("search does not support async")   

def build_agent():
    """
    Build a culinary agent.

    Returns:
        Agent: A culinary agent with specified tools and settings.
    """
    tools = [IngredientPriceTool()]

    # Initialize the agent with tools and settings
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_MULTI_FUNCTIONS,
        verbose=False,
    )
    return agent

def build_extraction_chain():
    """
    Build an extraction chain for culinary tasks.

    Returns:
        SimpleSequentialChain: A chain of processing steps for extracting culinary information.
    """
    extract_ingredients_template = """As a culinary expert tasked with extracting recommendations from the text, your mission is as follows:

    Recipe Name: {text}

    {format_instructions}
    """

    # Define a human message prompt template
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=extract_ingredients_template,
            input_variables=["text"],
            partial_variables={
                "format_instructions": parser_extraction.get_format_instructions()
            },
        )
    )

    # Create a chat prompt template
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

    # Create an extraction chain with the language model
    extract_recs_chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=False)

    # Build the agent and create a simple sequential chain
    _agent = build_agent()
    ss = SimpleSequentialChain(chains=[_agent, extract_recs_chain], verbose=False)
    return ss





In [13]:
def print_ingredients_with_color(ingredients):
    """
    Print a list of ingredients with colored formatting.

    Args:
        ingredients (list): A list of ingredient objects.

    Returns:
        None
    """
    for ingredient in ingredients:
        formatted_ingredient = f"{bcolors.OKBLUE}{ingredient.name} - {ingredient.quantity}{bcolors.ENDC}"
        print(formatted_ingredient)

def print_recommendations_with_color(search_query, recommendations):
    """
    Print search recommendations with colored formatting.

    Args:
        search_query (str): The search query.
        recommendations (list): A list of recommendation objects.

    Returns:
        None
    """
    # Print the search query in bold and blue
    print(f"{bcolors.OKBOLDBLUE}{search_query}{bcolors.ENDC}")  # Bold blue text
    
    # Iterate through recommendations and print them in yellow
    for recommendation in recommendations.recommendations:
        name = recommendation.name
        description = recommendation.description
        price = recommendation.price
        print(f"{bcolors.OKYELLOW}{name} - Description: {description} - Price: {price} {bcolors.ENDC}")  # Yellow text
    
    # Reset text formatting
    print(bcolors.ENDC)  # Reset text formatting to default

### Recipe Recommendation and Ingredient Listing <a class="anchor" id="Recipe-Recommendation"></a>

This script is designed to provide recipe recommendations and list ingredients based on user input. It utilizes a conversational AI model and a data retrieval system to create an interactive culinary experience. Here's how it works:

- The script begins by initializing a chain for generating ingredients and recommendations.
- Users are welcomed with a message prompting them to ask for a recipe or start a new search.

#### Input and Processing <a class="anchor" id="Input-and-Processing"></a>

- Users can input a recipe request or start a new search by typing their query.
- The script then runs the ingredient chain to retrieve a list of ingredients for the requested recipe.
- The retrieved ingredients are displayed to the user.

#### Recommendation Engine <a class="anchor" id="Recommendation-Engine"></a>

- Next, the script builds an extraction chain to gather recommendations for each ingredient from a store.
- It provides recommendations for available products based on the ingredients listed.
- These recommendations are displayed to the user.

#### Recipe Instructions <a class="anchor" id="Recipe-Instructions"></a>

- In addition to ingredients and recommendations, the script also provides recipe instructions.
- The user receives step-by-step instructions for preparing the requested dish.

#### Interaction and Exit <a class="anchor" id="Interaction-and-Exit"></a>

- After displaying the ingredients, recommendations, and instructions, the script prompts the user for the next action.
- Users can ask for another recipe, start a new search, or exit by using the "CTRL-D" shortcut (EOF).

This script combines the power of conversational AI, data retrieval, and culinary expertise to enhance the user's cooking experience. It's a handy tool for finding recipes and making informed ingredient choices. Enjoy exploring new dishes and flavors!

In [ ]:
if __name__ == "__main__":
    # Initialize the ingredient chain
    generate_ingredients_chain = build_chain()
    
    # Print welcome messages
    print(bcolors.OKCYAN + "Ask for any recipe, start a new search." + bcolors.ENDC)

    while True:
        try:
            query = input("> ").strip().lower().replace("new search:", "")

            # Run the ingredient chain to get ingredients
            result = generate_ingredients_chain.run(query)
            result = parser_ingredient.parse(result)
            ingredients = result.ingredients

            # Print ingredients
            print(bcolors.OKBOLDBLUE + "INGREDIENTS \n" + bcolors.ENDC)
            print_ingredients_with_color(ingredients=ingredients)

            # Build the extraction chain
            extract_agent = build_extraction_chain()

            # Print recommendations for each ingredient
            print(bcolors.OKBOLDBLUE + "\n\n RECOMMENDATIONS \n" + bcolors.ENDC)
            for ingredient in ingredients:
                sub_query = f"{ingredient.name} - {ingredient.quantity}"
                response = extract_agent.run(sub_query)
                response = parser_extraction.parse(response)
                print_recommendations_with_color(sub_query, response)

            # Print instructions
            print(bcolors.OKBOLDBLUE + "\n\n INSTRUCTIONS \n" + bcolors.ENDC)
            for instruction in result.instruction:
                print(bcolors.OKYELLOW + instruction + bcolors.ENDC)

            # Print options for the next query
            print(bcolors.OKCYAN + "Ask for any recipe, start a new search, or CTRL-D to exit." + bcolors.ENDC)
            print(bcolors.ENDC)
            print(">", end=" ", flush=True)

            print(bcolors.OKBLUE + "Goodbye!" + bcolors.ENDC)
        except EOFError:
        # Exit the loop when CTRL-D (EOF) is detected
            break


Ask for any recipe, start a new search.


>  hamburger


INGREDIENTS 

Ground beef - 1 pound
Hamburger buns - 4
Lettuce - 4 leaves
Tomato - 1
Onion - 1
Cheese - 4 slices
Salt - 1 teaspoon
Pepper - 1/2 teaspoon
Ketchup - to taste
Mustard - to taste
Pickles - to taste


 RECOMMENDATIONS 

Ground beef - 1 pound
Ground beef brochettes nature socopa - Description: 4 - 400g tray - Price: 5.45 
Brie good mayennay - Description: 1 kg cheese - Price: 9.29 

Hamburger buns - 4
Beef Tab - Description: 16 pieces of 150g each. - Price: 79.8 
Tomm of the Dauphiné Etoile du Vercors - Description: 6 cheeses, 400g total. - Price: 6.51 

Lettuce - 4 leaves
Green Lettuce Salad - Description: A refreshing green lettuce salad. - Price: 1.0395 
Energy Drink without Sugar (Red Bull) - Description: A pack of 4 cans of 25cl each. - Price: 5.775 

Tomato - 1
Ancient Tomatoes - Description: 750g tray - Price: 6.39 
Integer Chicken Simpl - Description: Chicken - Price: 6.52 

Onion - 1
onions organic organic organic - Description: 1kg net - Price: 2.82 
lemon carrierin